In [2]:
import argparse
import torch
import torch.optim as optim
from model.casRel import CasRel
from model.callback import MyCallBack
from model.data import load_data, get_data_iterator
from model.config import Config
from model.evaluate import metric
import torch.nn.functional as F
from fastNLP import Trainer, LossBase

seed = 226
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

parser = argparse.ArgumentParser(description='Model Controller')
parser.add_argument('--lr', type=float, default=1e-5, help='learning rate')
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--max_epoch', type=int, default=10)
parser.add_argument('--max_len', type=int, default=300)
parser.add_argument('--dataset', default='baidu', type=str, help='define your own dataset names')
parser.add_argument("--bert_name", default='./pretrained_models/bert-base-chinese/', type=str, help='choose pretrained bert name')
parser.add_argument('--bert_dim', default=768, type=int)
args = parser.parse_args(args=[])
con = Config(args)


def loss_fn(pred, gold, mask):
    pred = pred.squeeze(-1)
    loss = F.binary_cross_entropy(pred, gold, reduction='none')
    if loss.shape != mask.shape:
        mask = mask.unsqueeze(-1)
    loss = torch.sum(loss * mask) / torch.sum(mask)
    return loss

def get_loss(predict, target):  # Casrel计算loss的方程
    mask = target['mask']
    return loss_fn(predict['sub_heads'], target['sub_heads'], mask) + \
            loss_fn(predict['sub_tails'], target['sub_tails'], mask) + \
            loss_fn(predict['obj_heads'], target['obj_heads'], mask) + \
            loss_fn(predict['obj_tails'], target['obj_tails'], mask)

# if __name__ == '__main__':
model = CasRel(con).to(device)
data_bundle, rel_vocab = load_data(con.train_path, con.dev_path, con.test_path, con.rel_path)
train_dataset = get_data_iterator(con, data_bundle.get_dataset('train'), rel_vocab)
dev_dataset = get_data_iterator(con, data_bundle.get_dataset('dev'), rel_vocab, is_test=True)
test_dataset = get_data_iterator(con, data_bundle.get_dataset('test'), rel_vocab, is_test=True)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=con.lr)
# trainer = Trainer(train_data=train_data, model=model, optimizer=optimizer, batch_size=con.batch_size,
#                     n_epochs=con.max_epoch, loss=MyLoss(), print_every=con.period, use_tqdm=True,
#                     callbacks=MyCallBack(dev_data, rel_vocab, con))
#     trainer.train()
#     print("-" * 5 + "Begin Testing" + "-" * 5)
#     metric(test_data, rel_vocab, con, model)


Some weights of the model checkpoint at ./pretrained_models/bert-base-chinese/ were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./pretrained_models/bert-base-chinese/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from tqdm import tqdm
def train_epoch(train_loader, model, optimizer, epoch):
    global train_data, batch_samples
    # set model to training mode
    model.train() # 不固定batch normalization和dropout，需要更新
    # step number in one epoch: 336
    train_losses = 0
    for idx, batch_samples in enumerate(tqdm(train_loader)):  # tqdm是显示进度条的,每次加载一个batch32个数据
        train_data = batch_samples[0]
        target_data = batch_samples[1]
        predict = model(train_data['token_ids'], train_data['mask'], 
                train_data['sub_head'], train_data['sub_tail'])
        loss = get_loss(predict, target_data)
        # compute model output and loss
        train_losses += loss.item()
        # clear previous gradients, compute gradients of all variables wrt loss
        model.zero_grad()
        loss.backward()
        # gradient clipping
        # performs updates using calculated gradients
        optimizer.step()  # 要先Loss.backward()之后再用step,先清零参数空间的梯度,用了才会更新模型
        print(idx, train_losses)
    train_loss = float(train_losses) / len(train_loader)
    print("Epoch: {}, train loss: {}".format(epoch, train_loss))

best_val_f1 = 0.0
patience_counter = 0
# start training
epoch_num = 5
for epoch in range(1, epoch_num + 1):
    train_epoch(train_dataset, model, optimizer, epoch)
    # val_metrics = evaluate(dev_loader, model, mode='dev')
    # val_f1 = val_metrics['f1']
    # val_p = val_metrics['p']
    # val_r = val_metrics['r']
#     logging.info("Epoch: {}, dev loss: {}, f1 score: {}, precision: {}, recall: {}".format(epoch, val_metrics['loss'], val_f1, val_p, val_r))
#     improve_f1 = val_f1 - best_val_f1
#     if improve_f1 > 1e-5:
#         best_val_f1 = val_f1
#         model.save_pretrained(model_dir)
#         logging.info("--------Save best model!--------")
#         if improve_f1 < config.patience:
#             patience_counter += 1
#         else:
#             patience_counter = 0
#     else:
#         patience_counter += 1
#     # Early stopping and logging best f1
#     if (patience_counter >= config.patience_num and epoch > config.min_epoch_num) or epoch == config.epoch_num:
#         logging.info("Best val f1: {}".format(best_val_f1))
#         break
# logging.info("Training Finished!")

  2%|▏         | 1/64 [00:03<04:04,  3.88s/it]

0 27.29383087158203


  3%|▎         | 2/64 [00:06<03:10,  3.08s/it]

1 52.47056198120117


  5%|▍         | 3/64 [00:08<02:40,  2.63s/it]

...

61 56.8251610994339


 98%|█████████▊| 63/64 [02:25<00:02,  2.26s/it]

62 57.54821187257767


100%|██████████| 64/64 [02:26<00:00,  2.28s/it]

63 58.48323893547058
Epoch: 5, train loss: 0.9138006083667278


In [4]:
from model.evaluate import metric
metric(dev_dataset, rel_vocab, con, model)

100%|██████████| 36/36 [00:02<00:00, 13.10it/s]

correct_num:   0, predict_num:   0, gold_num:  63
f1: 0.00, precision: 0.00, recall: 0.00


(0.0, 0.0, 0.0)

In [20]:
from transformers import BertTokenizer
orders = ['subject', 'relation', 'object']
correct_num, predict_num, gold_num = 0, 0, 0
tokenizer = BertTokenizer.from_pretrained(con.bert_name)

for batch_x, batch_y in tqdm(dev_dataset):  # x用来测试，y是准确的数据集
    with torch.no_grad():
        token_ids = batch_x['token_ids']
        mask = batch_x['mask']
        encoded_text = model.get_encoded_text(token_ids, mask)
        pred_sub_heads, pred_sub_tails = model.get_subs(encoded_text)  # 预测
        sub_heads = torch.where(pred_sub_heads[0] > 0.05)[0]
        # if len(sub_heads)>0:
        #     print(sub_heads)
        sub_tails = torch.where(pred_sub_tails[0] > 0.5)[0]
        subjects = []
        for sub_head in sub_heads:
            sub_tail = sub_tails[sub_tails >= sub_head]
            if len(sub_tail) > 0:
                sub_tail = sub_tail[0]
                subject = ''.join(tokenizer.decode(token_ids[0][sub_head: sub_tail + 1]).split())
                subjects.append((subject, sub_head, sub_tail))

100%|██████████| 36/36 [00:03<00:00, 11.06it/s]


[]

In [94]:
optimizer.step()

In [3]:
from random import choice
from transformers import BertTokenizer
from transformers import BertModel
from collections import defaultdict
def find_head_idx(source, target):
    target_len = len(target)
    for i in range(len(source)):
        if source[i: i + target_len] == target:
            return i
    return -1

tokenizer = BertTokenizer.from_pretrained(con.bert_name)
bert = BertModel.from_pretrained(con.bert_name)

json_data = data_bundle.get_dataset('train')[223]
tokenized = tokenizer(json_data['text'])
tokens = tokenized['input_ids'] # 句子的length
masks = tokenized['attention_mask']
text_len = len(tokens)

token_ids = torch.tensor(tokens, dtype=torch.long)
masks = torch.tensor(masks, dtype=torch.bool)
"""主体和客体起始位置的记录"""
sub_heads, sub_tails = torch.zeros(text_len), torch.zeros(text_len)
sub_head, sub_tail = torch.zeros(text_len), torch.zeros(text_len)
obj_heads = torch.zeros((text_len, con.num_relations))
obj_tails = torch.zeros((text_len, con.num_relations))

s2ro_map = defaultdict(list)  # 创建一个dictionary，将键-值对更新为键-列表对，每个键可以调用list的属性
for spo in json_data['spo_list']:
    triple = (tokenizer(spo['subject'], add_special_tokens=False)['input_ids'], 
                rel_vocab.to_index(spo['predicate']),
                tokenizer(spo['object'], add_special_tokens=False)['input_ids']) # 把文本转换成id然后记录三元组,同时避免加入[CLS][SEP]这些特殊符号
    """
    - ISSUE: 如果某一个词语多次出现则只能找到第一个位置，这里有问题
    - SOLUTION: 再加一个变量记录这个词语是否出现过，如果出现过就记录他的位置，然后从这个位置开始往后找
    - WHY: 其实不改也可以，因为一个主体在文本中的意思应该是一样的。但是因为BERT模型会考虑前后文信息所以最好还是改一下？不确定，看验证结果，不是很严重的问题。
    """
    sub_head_idx = find_head_idx(tokens, triple[0])
    obj_head_idx = find_head_idx(tokens, triple[2])
    """可以试一下assert判断+终止"""
    if sub_head_idx != -1 and obj_head_idx != -1:
        sub = (sub_head_idx, sub_head_idx + len(triple[0]) - 1) # 主体位置
        s2ro_map[sub].append(
            (obj_head_idx, obj_head_idx + len(triple[2]) - 1, triple[1]))  # 用append解决一个主体对应多个客体的问题。客体位置+关系

if s2ro_map:  # 可能没有记录
    for s in s2ro_map:
        sub_heads[s[0]] = 1
        sub_tails[s[1]] = 1
    sub_head_idx, sub_tail_idx = choice(list(s2ro_map.keys()))
    sub_head[sub_head_idx] = 1
    sub_tail[sub_tail_idx] = 1
    for ro in s2ro_map.get((sub_head_idx, sub_tail_idx), []):
        obj_heads[ro[0]][ro[2]] = 1
        obj_tails[ro[1]][ro[2]] = 1

Some weights of the model checkpoint at ./pretrained_models/bert-base-chinese/ were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./pretrained_models/bert-base-chinese/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
